In [1]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl

from pyballistics import ozvb_lagrange, get_db_powder, get_powder_names
from joblib import Parallel, delayed
from matplotlib import cm
from matplotlib.colors import ListedColormap

# 1. Ballistic

In [2]:
d = 125*1e-3          # калибр м
q = 7.05           # вес снаряда кг
velocity_pm = 1700         # дульная скорость снаряда
n_s = 1           # нарезное орудие
max_pressure = 600*1e6    # максимальное давление Па
tube_lenght = 5543*1e-3      # длина трубы     
p_fors = 10*1e6  

SyntaxError: 'return' outside function (3161560940.py, line 8)

In [3]:
init_dict = {
   'powders': 
    [
       {'omega': None if wq is None else wq*q,
       'dbname': dbname}
    ],
    'init_conditions': 
    {
       'q': q,
       'd': d,
       'W_0': None if ro is None else wq*q/ro,
       'phi_1': 1.0,
       'p_0': p_fors, 
       'n_S': n_s
    },
    'igniter': 
    {
       'p_ign_0': 5_000_000.0 #check it 
    },
  
    'meta_lagrange': 
    {
       'CFL': 0.9, 
       'n_cells': 300
    },
    'stop_conditions': 
    {
       'x_p': tube_lenght,
       'steps_max': 8000,   
       't_max': 0.08,
       'p_max': max_pressure,
       'v_p': velocity_pm,
    }
    }

NameError: name 'wq' is not defined